In [ ]:
from cdcqr.data.deribit.config import get_deribit_sample_data_url
from cdcqr.data.deribit.data_utils import url_csv_zip_file_reader
from cdcqr.analytics.derivatives.option_utils import euro_vanilla
from cdcqr.analytics.derivatives.option_utils import find_vol
from cdcqr.common.config import SECONDS_PER_DAY
from cdcqr.analytics.derivatives.option_utils import euro_vanilla, Put
import cdcqr.data.deribit.data_utils as datautils

## Load Deribit sample data

In [ ]:
url = get_deribit_sample_data_url('OPTIONS', 'options_chain')
df = url_csv_zip_file_reader(url)

In [ ]:
df.head(2).T

In [ ]:
df1 = df.pipe(datautils.DeribitUtils.parse_optSymbol_col).pipe(datautils.DeribitUtils.parse_time_col)

In [ ]:
all_symbols = list(df1['symbol'].unique())
all_symbols

In [ ]:
dates = sorted(list(set([x.split('-')[1] for x in all_symbols])))

In [ ]:
#df['timestamp_s'], df['microsec'] = zip(*df['timestamp'].apply(DataTimeUtils.epoch_time_converter, 1))
#df['exp_timestamp_s'], df['exp_microsec'] = zip(*df['expiration'].apply(DataTimeUtils.epoch_time_converter, 1))

In [ ]:
df1['underlying_index'].unique()

In [ ]:
opt1 = df.iloc[1,:]
opt1

### Verify BS formula

In [ ]:
T = (opt1['expiration'] - opt1['timestamp'])/(1e6*SECONDS_PER_DAY*365)
T

In [ ]:
target_value = opt1['bid_price']
S = opt1['underlying_price']
K = opt1['strike_price']
sigma = opt1['ask_iv']/100
r = 0 

In [ ]:
print('S={},'.format(S), 'K={},'.format(K), 'T={},'.format(T), 'r={},'.format(r), 'sgima={},'.format(sigma))

In [ ]:
put_ask_iv = euro_vanilla(S, K, T, r, sigma, option='put')

In [ ]:
put_ask_iv, opt1['ask_price']*opt1['underlying_price']

### chceck greeks using opstrat

In [ ]:
Put.Price?

In [ ]:
Put.Price(S, K, r, sigma, T)

In [ ]:
Put.Delta(S, K, r, sigma, T)

In [ ]:
Put.Gamma(S, K, r, sigma, T)

In [ ]:
Put.Vega(S, K, r, sigma, T)/100

In [ ]:
Put.Theta(S, K, r, sigma, T)/365

In [ ]:
Put.Rho(S, K, r, sigma, T)/100

In [ ]:
delta                          -0.61919
gamma                           0.00109
vega                            2.78342
theta                         -28.40972
rho                            -0.37467

### snapshot example from 30SEP2021

In [ ]:
bid = 0.0095
ask = 0.0105

IV_bid = 0.854
IV_ask = 0.874
bid_price = 400.75
ask_price = 421.84
S = 42166
K = 60000
r=0
T = 30/365
sigma = IV_bid

In [ ]:
print('S={},'.format(S), 'K={},'.format(K), 'T={},'.format(T), 'r={},'.format(r), 'sgima={},'.format(sigma))

In [ ]:
call_price_IV_bid = euro_vanilla(S, K, T, r, IV_bid, option='call')
print('call price using IV bid ({}) =  {} '.format(IV_bid, call_price_IV_bid))
print('call bid price {}'.format(bid_price))

call_price_IV_ask = euro_vanilla(S, K, T, r, IV_ask, option='call')
print('call price using IV ask ({}) =  {} '.format(IV_ask, call_price_IV_ask))
print('call ask price {}'.format(ask_price))

In [ ]:
IV_bid = find_vol(bid_price, S, K, T, r)
IV_ask = find_vol(ask_price, S, K, T, r)

In [ ]:
IV_bid, IV_ask